In [31]:
HarryPotter_txt = open("/kaggle/input/harrypotter/harry potter.txt", "r").read()
HarryPotter_len = len(HarryPotter_txt)
print(HarryPotter_txt[:500])
print("\nLength:", HarryPotter_len)

Hagrid raised a gigantic fist and knocked three times on the castle door. The door swung open at once. A tall, black-haired witch in emerald-green robes stood there. She had a very stern face and Harry's first thought was that this was not someone to cross. 'The firs'-years, Professor McGonagall,' said Hagrid. Thank you, Hagrid. I will take them from here. She pulled the door wide. The Entrance Hall was so big you could have fitted the whole of the Dursleys' house in it. The stone walls were lit

Length: 3303


In [32]:
import string

all_characters = string.printable
num_characters = len(all_characters)

print(all_characters)
print("length:", num_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

length: 100


In [33]:
for char in list(set(HarryPotter_txt)):
    if char in all_characters:
        pass
    else:
        print(char)

In [34]:
import random
text_length = 600

def get_random_text():
    start_idx = random.randint(0, HarryPotter_len - text_length)
    end_idx = start_idx + text_length + 1
    return HarryPotter_txt[start_idx:end_idx]

print(get_random_text())

. 'The four houses are called Gryffindor, Hufflepuff, Ravenclaw and Slytherin. Each house has its own noble history and each has produced outstanding witches and wizards. While you are at Hogwarts, your triumphs will earn your house points, while any rule-breaking will lose house points. At the end of the year, the house with the most points is awarded the House Cup, a great honour. I hope each of you will be a credit to whichever house becomes yours. The Sorting Ceremony will take place in a few minutes in front of the rest of the school. I suggest you all smarten yourselves up as much as you 


In [35]:
import torch
from torch import nn, optim, functional

def str2tensor(text):
    tensor = torch.zeros(len(text)).long()
    for i, char in enumerate(text):
        tensor[i] = all_characters.index(char)
    return tensor

print(str2tensor("Gryffindor"))

tensor([42, 27, 34, 15, 15, 18, 23, 13, 24, 27])


In [36]:
def get_random_training_data():
    random_text = get_random_text()
    input_tensor = str2tensor(random_text[:-1])
    target_tensor = str2tensor(random_text[1:])
    return input_tensor, target_tensor

print(get_random_training_data())

(tensor([14, 94, 15, 18, 27, 28, 29, 74, 34, 14, 10, 27, 28, 94, 18, 23, 29, 24,
        94, 10, 94, 28, 22, 10, 21, 21, 94, 14, 22, 25, 29, 34, 94, 12, 17, 10,
        22, 11, 14, 27, 94, 24, 15, 15, 94, 29, 17, 14, 94, 17, 10, 21, 21, 75,
        94, 55, 17, 14, 34, 94, 12, 27, 24, 32, 13, 14, 13, 94, 18, 23, 73, 94,
        28, 29, 10, 23, 13, 18, 23, 16, 94, 27, 10, 29, 17, 14, 27, 94, 12, 21,
        24, 28, 14, 27, 94, 29, 24, 16, 14, 29, 17, 14, 27, 94, 29, 17, 10, 23,
        94, 29, 17, 14, 34, 94, 32, 24, 30, 21, 13, 94, 30, 28, 30, 10, 21, 21,
        34, 94, 17, 10, 31, 14, 94, 13, 24, 23, 14, 73, 25, 14, 14, 27, 18, 23,
        16, 94, 10, 11, 24, 30, 29, 94, 23, 14, 27, 31, 24, 30, 28, 21, 34, 75,
        68, 58, 14, 21, 12, 24, 22, 14, 94, 29, 24, 94, 43, 24, 16, 32, 10, 27,
        29, 28, 73, 68, 94, 28, 10, 18, 13, 94, 51, 27, 24, 15, 14, 28, 28, 24,
        27, 94, 48, 12, 42, 24, 23, 10, 16, 10, 21, 21, 75, 94, 68, 55, 17, 14,
        94, 28, 29, 10, 27, 29, 74, 24,

In [37]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size)

In [38]:
hidden_size = 256
num_layers = 1
learning_rate = 5e-3
input_size = num_characters
output_size = num_characters

rnn = RNN(input_size, hidden_size, output_size, num_layers)
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [39]:
rnn

RNN(
  (encoder): Embedding(100, 256)
  (gru): GRU(256, 256)
  (decoder): Linear(in_features=256, out_features=100, bias=True)
)

In [40]:
def train(input, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    for i in range(chunk_length):
        output, hidden = rnn(input[i].view(1, -1), hidden)
        loss += criterion(output.view(1, -1), target[i].view(1))

    loss.backward()
    optimizer.step()
    return loss.data.item() / chunk_length

In [41]:
def evaluate(seed_text="wh", prediction_length=100, temperature=0.8):
    hidden = rnn.init_hidden()
    start_input = str2tensor(seed_text)
    predicted = seed_text
    
    for i in range(len(seed_text) - 1):
        _, hidden = rnn(start_input[i], hidden)
    
    input = start_input[-1]
    
    for i in range(prediction_length):
        output, hidden = rnn(input, hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        input = str2tensor(predicted_char)
    
    return predicted

In [42]:
import time

start_time = time.time()
losses = []
avg_loss = 0

epochs = 1000
print_every = 100
log_every = 10

for epoch in range(1, epochs+1):
    input, target = get_random_training_data()
    loss = train(input, target)
    avg_loss += loss

    if epoch % print_every == 0:
        print(f"Epoch: {epoch} [time: {time.time() - start_time:3.0f}s] [Loss: {loss:.2f}]")
        print(evaluate("My", 100), "\n")

    if epoch % log_every == 0:
        losses.append(avg_loss / log_every)
        avg_loss = 0

Epoch: 100 [time:  74s] [Loss: 0.74]
My with time flaming about all you,' said Professor McGonagall have claswaking.' Her hearh touse point 

Epoch: 200 [time: 148s] [Loss: 0.09]
Myon Neville's cloak, which was fastend under houses. The Sorting Ceremony will take your seats in the 

Epoch: 300 [time: 222s] [Loss: 0.05]
Myow and spend free time in your house common ready to the Dursleys saying that home to the rest of th 

Epoch: 400 [time: 296s] [Loss: 0.02]
Myon Neville's cloak, which was fastened under his left ear, and on Ron's smudged nose. Harry nervous, 

Epoch: 500 [time: 371s] [Loss: 0.02]
Mycelves wich en he'd learnt and wondering which one she'd need. Harry tried here door floors. They fo 

Epoch: 600 [time: 446s] [Loss: 0.04]
My will take place in a few minutes in front of the rest of the school. I suggest you all smarten your 

Epoch: 700 [time: 520s] [Loss: 0.03]
My crowded in, standing rather closer together that the Dursleys' house in it. The stone walls were li 

Epoch: